In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Homework 9
Let's predict the future.

## Part 0: Setup

In [2]:
import tensorflow as tf
import numpy as np
import util

# Colors to visualize the labeling
COLORS = np.array([(0,0,0), (255,0,0), (0,255,0), (255,255,0), (0,0,255), (255,255,255)], dtype=np.uint8)
CROP_SIZE = 64
N_ACTION = 5

offsets = [0, 6, 15, 30, 60, 120]

def parser(record):
    # Parse the TF record
    
    feature ={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'channels': tf.FixedLenFeature([], tf.int64),
        'n_future': tf.FixedLenFeature([], tf.int64),
        'action': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string)
    }
    values = {'position' : np.float32, 'is_dying': np.int64, 'on_ground': np.int64, 'coins': np.int64}
    for k,v in enumerate(values):
        t = values[v]
        for j, o in enumerate(offsets):
            feature[v+'_%d'%j] = tf.FixedLenFeature([], t)

    parsed = tf.parse_single_example(record, features=feature)
     
    # Load the data and format it
    W = tf.cast(parsed['width'], tf.int32)
    H = tf.cast(parsed['height'], tf.int32)
    C = tf.cast(parsed['channels'], tf.int32)
    A = tf.cast(parsed['action'], tf.int32)
    actions = tf.stack([tf.bitwise.bitwise_and(A, (1<<i)) > 0 for i in range(N_ACTION)])
    image = tf.reshape(tf.decode_raw(parsed["image_raw"], tf.uint8), [H,W,C])
    
    current_position = tf.cast(parsed['position_0'], tf.float32)
    current_is_dying = tf.cast(parsed['is_dying_0'], tf.bool)
    current_coins = tf.cast(parsed['coins_0'], tf.float32)
    
    future_position = tf.stack([tf.cast(parsed['position_%d'%o], tf.float32) for o in range(1,len(offsets))])
    future_is_dying = tf.stack([tf.cast(parsed['is_dying_%d'%o], tf.bool) for o in range(1,len(offsets))])
    future_coins = tf.stack([tf.cast(parsed['coins_%d'%o], tf.float32) for o in range(1,len(offsets))])
    
    ## No data augmentation this time, as it might affect the future
    return image, actions, current_position, current_is_dying, current_coins, future_position, future_is_dying, future_coins

def load_dataset(tfrecord):
    # Load the dataset
    dataset = tf.contrib.data.TFRecordDataset(tfrecord)

    # Parse the tf record entries
    dataset = dataset.map(parser, num_threads=8, output_buffer_size=1024)

    # Shuffle the data, batch it and run this for multiple epochs
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(32)
    dataset = dataset.repeat()
    return dataset

## Part 1: Define your network

In [116]:
# Create a new log directory (if you run low on disk space you can either disable this or delete old logs)
# run: `tensorboard --logdir log` to see all the nice summaries
for n_model in range(1000):
    LOG_DIR = 'log/model_%d'%n_model
    from os import path
    if not path.exists(LOG_DIR):
        break

# Lets clear the tensorflow graph, so that you don't have to restart the notebook every time you change the network
tf.reset_default_graph()

TF_COLORS = tf.constant(COLORS)

train_data = load_dataset('future_train.tfrecord')
valid_data = load_dataset('future_val.tfrecord')

# Create an iterator for the datasets
# The iterator allows us to quickly switch between training and validataion
iterator = tf.contrib.data.Iterator.from_structure(train_data.output_types, ((None,64,64,9), (None,N_ACTION), (None,), (None,), (None,), (None,len(offsets)-1), (None,len(offsets)-1), (None,len(offsets)-1)))

# and fetch the next images from the dataset (every time next_image is evaluated a new image set of 32 images is returned)
image, action, current_position, current_is_dying, current_coins, future_position, future_is_dying, future_coins = iterator.get_next()

# Define operations that switch between train and valid
switch_train_op = iterator.make_initializer(train_data)
switch_valid_op = iterator.make_initializer(valid_data)

# Convert the input and label
image = tf.identity(image, name='images')
image = tf.cast(image, tf.float32)
action = tf.identity(tf.cast(action, tf.int32), name='action')

current_position = tf.identity(current_position, name='current_position')
current_is_dying = tf.identity(current_is_dying, name='current_is_dying')
current_coins = tf.identity(current_coins, name='current_coins')

future_position = tf.identity(future_position, name='future_position')
future_is_dying = tf.identity(future_is_dying, name='future_is_dying')
future_coins = tf.identity(future_coins, name='future_coins')

# Whiten the image
white_image = (image - 100.) / 72.
h = white_image

# TODO: Define your convnet and loss here
# In preparation for the next homework you might want to make this network small and efficient.
training = tf.placeholder_with_default(False, (), name='training')

# Build the network out of a few convolutional layers
h = tf.contrib.layers.conv2d(h, 5, (5,5), stride=1, weights_regularizer=tf.nn.l2_loss)
h = tf.layers.batch_normalization(h, center=False, scale=False, training=training)   
h = tf.contrib.layers.conv2d(h, 10, (5,5), stride=1, weights_regularizer=tf.nn.l2_loss)
h = tf.layers.batch_normalization(h, center=False, scale=False, training=training)
h = tf.contrib.layers.conv2d(h, 9, (5,5), stride=1, weights_regularizer=tf.nn.l2_loss)
h = tf.layers.batch_normalization(h, center=False, scale=False, training=training)

# Hook up a fully connected layer to predict the action
fc = tf.contrib.layers.fully_connected(h, 9, activation_fn=None)

# Combine the action and the output of the conv layer
h = tf.add(fc, h)

output = tf.identity(h, name='output')
# Use a cross entropy for the action and is_dying, L2 for position and coin.
# sigmoid for action and is dying
# TODO: define losses here
action_logit = tf.contrib.layers.conv2d(h, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
action_logit = tf.contrib.layers.conv2d(action_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
action_logit = tf.contrib.layers.conv2d(action_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
action_logit = tf.contrib.layers.conv2d(action_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
action_logit = tf.contrib.layers.conv2d(action_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
action_logit = tf.contrib.layers.conv2d(action_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
action_logit = tf.contrib.layers.flatten(action_logit)

future_is_dying_logit = tf.contrib.layers.conv2d(h, 5, (5,5), stride=64, weights_regularizer=tf.nn.l2_loss)
future_is_dying_logit = tf.contrib.layers.flatten(future_is_dying_logit)

future_position_logit = tf.contrib.layers.conv2d(h, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
future_position_logit = tf.contrib.layers.conv2d(future_position_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
future_position_logit = tf.contrib.layers.conv2d(future_position_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
future_position_logit = tf.contrib.layers.conv2d(future_position_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
future_position_logit = tf.contrib.layers.conv2d(future_position_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
future_position_logit = tf.contrib.layers.conv2d(future_position_logit, 5, (5,5), stride=2, weights_regularizer=tf.nn.l2_loss)
future_position_logit = tf.contrib.layers.flatten(future_position_logit)

future_coins_logit = tf.contrib.layers.conv2d(h, 5, (5,5), stride=64, weights_regularizer=tf.nn.l2_loss)
future_coins_logit = tf.contrib.layers.flatten(future_coins_logit)

coins_loss = tf.reduce_mean(tf.abs(future_coins_logit - future_coins))
position_loss = tf.reduce_mean(tf.abs(future_position_logit - future_position))
is_dying_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=future_is_dying_logit, labels=future_is_dying))
#action_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=action_logit, labels=action))
action = tf.cast(action, tf.float32)
action_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=action_logit, labels=action))

#output = tf.identity(h, name='output')

loss = 0.05*coins_loss + position_loss + is_dying_loss + action_loss

# a binary vector of size 5 for each image in the batch (DO NOT USE action TO PREDICT THIS!)
pred_action = tf.identity(action_logit > 0.5, name='predicted_action')

# vectors of size (5) for each image in the batch (use action AND current_position, current_is_dying, current_coins TO PREDICT THIS)
# Hint: for some variables you might want to make them relative to current_..., for others not
pred_is_dying = tf.identity(future_is_dying_logit > 0.5, name='predicted_is_dying')
pred_position = tf.identity(future_position_logit, name='predicted_position')
pred_coins = tf.identity(future_coins_logit, name='predicted_coins')

action_acc = tf.reduce_mean(tf.cast(tf.equal(tf.cast(pred_action, tf.float32), tf.cast(action, tf.float32)), tf.float32))
dying_acc = tf.reduce_mean(tf.cast(tf.equal(tf.cast(pred_is_dying, tf.float32), tf.cast(future_is_dying, tf.float32)), tf.float32))

# Let's weight the regularization loss down, otherwise it will hurt the model performance
# You can tune this weight if you wish
regularization_loss = tf.losses.get_regularization_loss()
total_loss = loss + 1e-6 * regularization_loss

# Adam will likely converge much faster than SGD for this assignment.
optimizer = tf.train.AdamOptimizer(0.001, 0.9, 0.999)

# use that optimizer on your loss function (control_dependencies makes sure any 
# batch_norm parameters are properly updated)
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    opt = optimizer.minimize(total_loss)

# Let's define some summaries for tensorboard
tf.summary.image('image1', image[:,:,:,:3], max_outputs=3)
tf.summary.image('image2', image[:,:,:,3:6], max_outputs=3)
tf.summary.image('image3', image[:,:,:,6:9], max_outputs=3)
tf.summary.scalar('action_loss', tf.placeholder(tf.float32, name='action_loss'))
tf.summary.scalar('is_dying_loss', tf.placeholder(tf.float32, name='is_dying_loss'))
tf.summary.scalar('position_loss', tf.placeholder(tf.float32, name='position_loss'))
tf.summary.scalar('coins_loss', tf.placeholder(tf.float32, name='coins_loss'))
tf.summary.scalar('loss', tf.placeholder(tf.float32, name='loss'))
tf.summary.scalar('val_loss', tf.placeholder(tf.float32, name='val_loss'))

merged_summary = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(LOG_DIR, tf.get_default_graph())

# Let's compute the model size
print( "Total number of variables used ", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]) )

Total number of variables used  15559


## Part 2: Training

Training might take up to 20 min depending on your architecture (and if you have a GPU or not).

In [119]:
# Start a session
sess = tf.Session()

# Set up training
sess.run(tf.global_variables_initializer())

# Run the training for some iterations
for it in range(500):
    sess.run(switch_train_op)

    loss_vals = []
    # Run 10 training iterations and 1 validation iteration
    for i in range(10):
        #sess.run(action_loss)
        #sess.run(is_dying_loss)
        #sess.run(position_loss)
        #sess.run(coins_loss)
        loss_val = sess.run([loss,action_loss,is_dying_loss,position_loss,coins_loss,opt])[:-1]
        loss_vals.append(loss_val)
    # Compute the summary
    mean_loss = np.mean(np.array(loss_vals), axis=0)
    summary = {n+':0':mean_loss[i] for i, n in enumerate(['loss','action_loss','is_dying_loss','position_loss','coins_loss'])}

    # Compute the validation loss
    sess.run(switch_valid_op)
    loss_val = sess.run(loss)
    summary['val_loss:0'] = loss_val
    
    summary_writer.add_summary( sess.run(merged_summary, summary), it )

    # Let's update tensorboard
    #action prediction 90% accuracy = 25pts
    #death prediction 100% accuracy = 25pts
    #pos prediction L2=0.25 = 25pts
    #coins prediction L2=0.01 = 25pts
    #print('[%3d] Loss: %0.3f  \t  val loss A.: %0.3f'%(it, mean_loss[0], loss_val))
    if (it%1 == 0):
        sess.run(switch_valid_op)
        action_ac, dying_ac, pos_loss, coin_loss = sess.run([action_acc, dying_acc, position_loss, coins_loss])
        print('L: ' + str(mean_loss[0]) + ' VL: '+ str(loss_val) + ' A: '+ str(action_ac)+' D: '+ str(dying_ac)+' P: '+ str(pos_loss)+' C: '+ str(coin_loss))
        if(action_ac > 0.85):
            name = 'assignment9a_' + str(int(100 * action_ac)) + '_' + str(int(1000 * pos_loss)) + '.tfg'
            util.save(name, session=sess)

L: 0.786344 VL: 0.757008 A: 0.825 D: 1.0 P: 0.0371452 C: 0.625108
L: 0.766376 VL: 0.72958 A: 0.86875 D: 1.0 P: 0.041512 C: 0.02636
L: 0.753649 VL: 0.744067 A: 0.85 D: 1.0 P: 0.0388828 C: 0.0
L: 0.747551 VL: 0.749057 A: 0.76875 D: 0.99375 P: 0.0478214 C: 0.0
L: 0.747945 VL: 0.768114 A: 0.76875 D: 1.0 P: 0.0551077 C: 0.0
L: 0.736389 VL: 0.741619 A: 0.8875 D: 1.0 P: 0.0238293 C: 0.0
L: 0.756095 VL: 0.727549 A: 0.85625 D: 1.0 P: 0.0316743 C: 0.0
L: 0.742134 VL: 0.719847 A: 0.85 D: 1.0 P: 0.0541572 C: 0.0
L: 0.738061 VL: 0.771067 A: 0.85 D: 1.0 P: 0.028917 C: 0.0
L: 0.734507 VL: 0.772427 A: 0.80625 D: 1.0 P: 0.0272039 C: 0.0
L: 0.742112 VL: 0.718529 A: 0.8375 D: 1.0 P: 0.029493 C: 0.0
L: 0.737246 VL: 0.730207 A: 0.8875 D: 0.99375 P: 0.0668796 C: 0.0
L: 0.742025 VL: 0.772011 A: 0.79375 D: 0.99375 P: 0.0388284 C: 0.0
L: 0.747275 VL: 0.785453 A: 0.85 D: 1.0 P: 0.0340083 C: 0.0
L: 0.754001 VL: 0.727422 A: 0.84375 D: 1.0 P: 0.0503584 C: 0.0
L: 0.742318 VL: 0.725503 A: 0.81875 D: 1.0 P: 0.0313389

L: 0.704 VL: 0.70137 A: 0.8875 D: 1.0 P: 0.0200565 C: 0.0
L: 0.6905 VL: 0.709872 A: 0.88125 D: 0.99375 P: 0.0133752 C: 0.0
L: 0.700526 VL: 0.780305 A: 0.84375 D: 1.0 P: 0.0368577 C: 0.0
L: 0.702893 VL: 0.675788 A: 0.9 D: 1.0 P: 0.0133759 C: 0.0
L: 0.71089 VL: 0.833526 A: 0.8625 D: 1.0 P: 0.0167987 C: 0.0
L: 0.729767 VL: 0.711194 A: 0.81875 D: 1.0 P: 0.0261756 C: 0.0
L: 0.724473 VL: 0.71956 A: 0.84375 D: 0.99375 P: 0.0302908 C: 0.0
L: 0.7159 VL: 0.756767 A: 0.85 D: 1.0 P: 0.0576279 C: 0.0
L: 0.70557 VL: 0.721408 A: 0.9 D: 1.0 P: 0.0254497 C: 0.0
L: 0.715081 VL: 0.824028 A: 0.85 D: 1.0 P: 0.0277366 C: 0.0
L: 0.712566 VL: 0.722891 A: 0.88125 D: 1.0 P: 0.0292751 C: 0.0
L: 0.711326 VL: 0.727314 A: 0.8625 D: 0.99375 P: 0.0446924 C: 0.0
L: 0.708368 VL: 0.730609 A: 0.85 D: 0.99375 P: 0.0264417 C: 0.0
L: 0.712971 VL: 0.728414 A: 0.86875 D: 1.0 P: 0.0325392 C: 0.0
L: 0.701005 VL: 0.74626 A: 0.8625 D: 1.0 P: 0.0158738 C: 0.0
L: 0.697337 VL: 0.699339 A: 0.9125 D: 1.0 P: 0.0133163 C: 0.0
L: 0.69544

KeyboardInterrupt: 

## Part 3: Evaluation
### Compute the validation accuracy

In [ ]:
sess.run(switch_valid_op)
action_ac, dying_ac, pos_loss, coin_loss = sess.run([action_acc, dying_acc, position_loss, coins_loss])

print('Action prediction accuracy: ' + str(action_ac))
print('Dying prediction accuracy: ' + str(dying_ac))
print('Position prediction L2: ' + str(pos_loss))
print('Coin prediction L2: ' + str(coin_loss))

## Part 4: Save Model
Please note that we also want you to turn in your ipynb for this assignment.  Zip up the ipynb along with the tfg for your submission.

In [ ]:
util.save('assignment9.tfg', session=sess)